In [38]:
import pandas as pd
import os
import numpy as np
import joblib

entorno = 'local'  # Elegir "VM" o "local" para correr en entorno local
ventana_input = 12
ventana_output = 2
ventana_test = 3
lags = [1, 2, 3, 6, 12]


# Configurar entorno
if entorno == 'VM':
    carpeta_datasets = os.path.expanduser('~/buckets/b1/datasets')
    carpeta_exp_base = os.path.expanduser('~/buckets/b1/exp')
elif entorno == 'local':
    carpeta_datasets = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Datos'
    carpeta_exp_base = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Resultados'
else:
    raise Exception("Entorno especificado incorrectamente")




In [22]:
# Cargar datos
sell_in = pd.read_csv(os.path.join(carpeta_datasets, 'sell-in.txt'), delimiter='\t')
tb_productos = pd.read_csv(os.path.join(carpeta_datasets, 'tb_productos_descripcion.txt'), delimiter='\t')
tb_stocks = pd.read_csv(os.path.join(carpeta_datasets, 'tb_stocks.txt'), delimiter='\t')
productos_predecir = pd.read_csv(os.path.join(carpeta_datasets, 'productos_a_predecir.txt'), delimiter='\t')



In [23]:
#Combinar datasets
dataset_completo = sell_in.merge(tb_productos, how='left', on='product_id')
dataset_completo



,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion
0,201701,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador
1,201701,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador
2,201701,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador
3,201701,10125,20524,0,1,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador
4,201701,10012,20524,0,11,1.54452,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2945813,201912,10105,20853,0,1,0.02230,0.02230,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,Sabor 1
2945814,201912,10092,20853,0,1,0.00669,0.00669,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,Sabor 1
2945815,201912,10006,20853,0,7,0.02898,0.02898,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,Sabor 1
2945816,201912,10018,20853,0,4,0.01561,0.01561,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,Sabor 1


In [24]:
#Convertir periodo en index
dataset_completo['Timestamp'] = pd.to_datetime(dataset_completo['periodo'], format='%Y%m')
dataset_completo.set_index('Timestamp', inplace=True)
dataset_completo.drop(columns=['periodo'], inplace=True)

#Filtramos los productos de interes para reducir el tamaño del dataset
dataset_completo = dataset_completo[dataset_completo['product_id'].isin(list(productos_predecir.values.reshape(-1)))]
dataset_completo.head()

,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion
Timestamp,,,,,,,,,,,,
2017-01-01,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador
2017-01-01,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador
2017-01-01,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador
2017-01-01,10125,20524,0,1,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador
2017-01-01,10012,20524,0,11,1.54452,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador


In [32]:
detalle_productos = dataset_completo[['product_id', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'descripcion']].copy()
detalle_productos = detalle_productos.drop_duplicates()

detalle_productos


,product_id,cat1,cat2,cat3,brand,sku_size,descripcion
Timestamp,,,,,,,
2017-01-01,20524,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador
2017-01-01,20311,HC,VAJILLA,Cristalino,Importado,1000.0,Abrillantador
2017-01-01,20654,PC,DEOS,Aero,DEOS2,60.0,Sabor 8
2017-01-01,20828,FOODS,SOPAS Y CALDOS,Dishes,MAGGI,8.0,Fideos al verdeo
2017-01-01,20941,HC,HOGAR,Paño,MUSCULO,1.0,Multiuso 1
...,...,...,...,...,...,...,...
2019-09-01,20962,FOODS,OTROS,Galletitas,HARINAMA,180.0,Sabor 8
2019-09-01,20975,FOODS,OTROS,Galletitas,HARINAMA,180.0,Sabor 2
2019-09-01,20995,FOODS,OTROS,Galletitas,HARINAMA,180.0,Sabor 1


In [33]:
# Fecha final del dataset completo
fecha_final = df_productos_filtrado.index.max()


df_producto_cliente_completo = pd.DataFrame()


for producto in dataset_completo['product_id'].unique():
    ventas_mes_por_producto = dataset_completo[dataset_completo['product_id'] == producto].copy()
    for cliente in ventas_mes_por_producto['customer_id'].unique():
        print(f'Procesando Producto {producto} - Cliente {cliente}',  end="\r")
        ventas_mes_por_producto_cliente = ventas_mes_por_producto[ventas_mes_por_producto['customer_id'] == cliente].copy()
        fecha_inicial = ventas_mes_por_producto_cliente.index.min()
        date_range = pd.date_range(start=fecha_inicial, end=fecha_final, freq='MS')
        ventas_mes_por_producto_cliente = ventas_mes_por_producto_cliente.reindex(date_range)
        ventas_mes_por_producto_cliente = ventas_mes_por_producto_cliente.reset_index().rename(columns= {'index': 'Timestamp'}) #Volvemos la columna Timestamp para poder hacer merge
        #Rellenar los valores faltantes generados
        ventas_mes_por_producto_cliente['tn'] = ventas_mes_por_producto_cliente['tn'].fillna(0)
        ventas_mes_por_producto_cliente['cust_request_qty'] = ventas_mes_por_producto_cliente['cust_request_qty'].fillna(0)
        ventas_mes_por_producto_cliente['cust_request_tn'] = ventas_mes_por_producto_cliente['cust_request_tn'].fillna(0)
        ventas_mes_por_producto_cliente['product_id'] = producto
        ventas_mes_por_producto_cliente['customer_id'] = cliente
        ventas_mes_por_producto_cliente['cat1'] = detalle_productos[detalle_productos['product_id'] == producto]['cat1'].iloc[0]
        ventas_mes_por_producto_cliente['cat2'] = detalle_productos[detalle_productos['product_id'] == producto]['cat2'].iloc[0]
        ventas_mes_por_producto_cliente['cat3'] =  detalle_productos[detalle_productos['product_id'] == producto]['cat3'].iloc[0]
        ventas_mes_por_producto_cliente['brand'] =  detalle_productos[detalle_productos['product_id'] == producto]['brand'].iloc[0]
        ventas_mes_por_producto_cliente['descripcion'] =  detalle_productos[detalle_productos['product_id'] == producto]['descripcion'].iloc[0]
        ventas_mes_por_producto_cliente['sku_size'] =  detalle_productos[detalle_productos['product_id'] == producto]['sku_size'].iloc[0]
        
        #Falta completar precios cuidados
        
        df_producto_cliente_completo = pd.concat([df_producto_cliente_completo, ventas_mes_por_producto_cliente], axis = 0)
df_producto_cliente_completo.to_csv(os.path.join(carpeta_datasets, 'df_producto_cliente_completo.csv'), index = False)

In [39]:
# Extraer el mes
df_producto_cliente_completo['mes'] = df_producto_cliente_completo['Timestamp'].dt.month
# Calcular el trimestre
df_producto_cliente_completo['quarter'] = df_producto_cliente_completo['mes'].apply(lambda x: (x-1)//3 + 1)
df_producto_cliente_completo['fin_quarter'] = df_producto_cliente_completo['mes'].apply(lambda x: 1 if x in [3,6,9,12] else 0)

In [41]:
def diff_in_months(row):
    fecha_ingreso = row['fecha_ingreso_producto']
    fecha_actual = row['Timestamp']

    # Calcular la diferencia en meses
    diff_months = (fecha_actual.year - fecha_ingreso.year) * 12 + (fecha_actual.month - fecha_ingreso.month)
    
    return diff_months



df_producto_cliente_completo['fecha_ingreso_producto'] = df_producto_cliente_completo.groupby('product_id')['Timestamp'].transform('min')
df_producto_cliente_completo['edad_producto'] = df_producto_cliente_completo.apply(diff_in_months, axis = 1)

df_producto_cliente_completo.drop(columns=['fecha_ingreso_producto'], inplace = True)
df_producto_cliente_completo.head()

,Timestamp,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,mes,quarter,fin_quarter,edad_producto
0,2017-01-01,10234,20524,0.0,2.0,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,1,1,0,0
1,2017-02-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,2,1,0,1
2,2017-03-01,10234,20524,0.0,1.0,0.01514,0.01514,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,3,1,1,2
3,2017-04-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,4,2,0,3
4,2017-05-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,5,2,0,4


In [42]:
ventas_cat1 = df_producto_cliente_completo.groupby(['Timestamp', 'customer_id', 'cat1'])['tn'].sum().reset_index().rename(columns={'tn': 'ventas_cat1'})
ventas_cat2 = df_producto_cliente_completo.groupby(['Timestamp', 'customer_id', 'cat1', 'cat2'])['tn'].sum().reset_index().rename(columns={'tn': 'ventas_cat2'})
ventas_cat3 = df_producto_cliente_completo.groupby(['Timestamp', 'customer_id', 'cat1', 'cat2', 'cat3'])['tn'].sum().reset_index().rename(columns={'tn': 'ventas_cat3'})
ventas_familia_productos = df_producto_cliente_completo.groupby(['Timestamp', 'customer_id', 'cat1', 'cat2', 'cat3', 'brand', 'descripcion'])['tn'].sum().reset_index().rename(columns={'tn': 'ventas_familia_producto'})

#Guardar las ventas agrupadas
joblib.dump(ventas_cat1, os.path.join(carpeta_datasets,'ventas_cat1.pkl'))
joblib.dump(ventas_cat2, os.path.join(carpeta_datasets,'ventas_cat2.pkl'))
joblib.dump(ventas_cat3, os.path.join(carpeta_datasets,'ventas_cat3.pkl'))
joblib.dump(ventas_familia_productos, os.path.join(carpeta_datasets,'ventas_familia_productos.pkl'))

['C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Datos\\ventas_familia_productos.pkl']

In [47]:
df_fe_familia_productos = df_producto_cliente_completo.merge(ventas_cat1, how='left', on=['Timestamp', 'customer_id', 'cat1'])
df_fe_familia_productos = df_fe_familia_productos.merge(ventas_cat2, how='left', on=['Timestamp', 'customer_id', 'cat1', 'cat2'])
df_fe_familia_productos = df_fe_familia_productos.merge(ventas_cat3, how='left', on=['Timestamp', 'customer_id', 'cat1', 'cat2', 'cat3'])
df_fe_familia_productos = df_fe_familia_productos.merge(ventas_familia_productos, how='left', on=['Timestamp', 'customer_id', 'cat1', 'cat2', 'cat3', 'brand', 'descripcion'])

df_fe_familia_productos

,Timestamp,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,...,sku_size,descripcion,mes,quarter,fin_quarter,edad_producto,ventas_cat1,ventas_cat2,ventas_cat3,ventas_familia_producto
0,2017-01-01,10234,20524,0.0,2.0,0.05300,0.05300,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,1,1,0,0,14.31686,4.96628,3.03194,0.25684
1,2017-02-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,2,1,0,1,2.14290,0.10339,0.00000,0.00000
2,2017-03-01,10234,20524,0.0,1.0,0.01514,0.01514,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,3,1,1,2,8.59237,2.23835,1.52777,0.04699
3,2017-04-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,4,2,0,3,9.18260,4.47157,2.35257,0.00000
4,2017-05-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,5,2,0,4,7.79714,0.50130,0.09348,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6910867,2019-12-01,10083,20127,0.0,1.0,0.24645,0.24645,HC,ROPA LAVADO,Liquido,...,3000.0,Regular,12,4,1,3,35.48098,20.96524,6.93741,2.10633
6910868,2019-12-01,10074,20127,0.0,1.0,20.70148,20.70148,HC,ROPA LAVADO,Liquido,...,3000.0,Regular,12,4,1,3,113.11048,64.95754,49.02482,22.42660
6910869,2019-12-01,10172,20127,0.0,1.0,2.58769,2.58769,HC,ROPA LAVADO,Liquido,...,3000.0,Regular,12,4,1,3,14.09831,11.78360,11.78360,4.31281
6910870,2019-12-01,10175,20127,0.0,1.0,0.36967,0.36967,HC,ROPA LAVADO,Liquido,...,3000.0,Regular,12,4,1,3,15.63015,7.85789,5.05363,0.73934


In [48]:
df_fe_familia_productos.to_csv(os.path.join(carpeta_datasets, 'df_producto_cliente_completo.csv'), index = False)

In [61]:
#Chequeamos la consistencia de los valores para el cliente 10001 vs dataset luego de merge

ventas_cat1[ventas_cat1['customer_id']==10001]

,Timestamp,customer_id,cat1,ventas_cat1
0,2017-01-01,10001,FOODS,320.42839
1,2017-01-01,10001,HC,1185.94819
2,2017-01-01,10001,PC,368.04266
3,2017-01-01,10001,REF,2.87670
1090,2017-02-01,10001,FOODS,374.20160
...,...,...,...,...
56146,2019-11-01,10001,REF,0.12012
57993,2019-12-01,10001,FOODS,329.71678
57994,2019-12-01,10001,HC,1607.98960
57995,2019-12-01,10001,PC,375.81468


In [66]:
dataset_completo[(dataset_completo['customer_id']==10001) & (dataset_completo.index == '2019-12-01')].groupby('cat1')['tn'].sum()

cat1
FOODS     329.71678
HC       1607.98960
PC        375.81468
REF         0.00255
Name: tn, dtype: float64